# State Conservation List - VIC

This notebook downloads the VIC lists from the [Victorian Government Open Data Portal](https://vba.dse.vic.gov.au/vba/downloadVSC.do) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/VIC` directory, process the lists and save them to `current-lists`.
The lists can be loaded into the lists-test environment then reviewed by following the instructions below.

## Lists in the ALA Species List tool
* Conservation list: __[dr655](https://lists.ala.org.au/speciesListItem/list/dr655)__ (dr655 in [test](https://lists-test.ala.org.au/speciesListItem/list/dr655))

* Collection: __[dr655](https://collections.ala.org.au/public/show/dr655)__
## Sources
Victorian Biodivesity Atlas - Victoria Department of Environment, Land, Water and Planning
See https://vba.biodiversity.vic.gov.au/vba

* __[Metadata (Open Data Portal)](https://discover.data.vic.gov.au/dataset/victorian-biodiversity-atlas-vba-taxa-list1)__
* __[VBA Metadata](https://vba.dse.vic.gov.au/vba/index.jsp)__ (requires sign in)
* __[Data](https://vba.dse.vic.gov.au/vba/downloadVSC.do)__

**Metadata Description**
This list is generated from the subset of species listed in the Victorian Biodiversity Atlas (VBA) Taxa List downloaded from https://vba.dse.vic.gov.au/vba/downloadVSC.do which have a conservation status attributed to them.

After the review of the Flora and Fauna Guarantee Act (FFG Act) all previously listed species under the three different legislations and policy have been re-assessed. The VBA has been updated to reflect the gazetted and published FFG Threatened List. The Conservation Status is now only EPBC and FFG. Vic Advisory Lists are now redundant but retained for your information as 'Old Advisory List'. This is the interim measure before the national Single Operational List is finalised, which will replace the 3 different lists previously held in Victoria. The FFG categories are the nationally agreed IUCN Red list terms plus the EPBC Conservation Dependent category.

The main attributes in this table are the current VBA taxon_id, scientific name, common name, authority, family (where relevant), conservation status, origin and taxon type. Where possible, taxon id's have been included that provide a match to the obsolete Atlas of Victorian Wildlife taxaid, and the Flora Infromation System.

**URL**
https://discover.data.vic.gov.au/dataset/victorian-biodiversity-atlas-vba-taxa-list1

Prerequisite: Data file has been downloaded into `source-data/VIC/Species-Checklist.csv`


In [11]:
import pandas as pd
import sys
import os
import datetime


# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
monthStr = datetime.datetime.now().strftime('%Y%m')

sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

cdataDir = "current-lists/conservation-lists/"
vicchecklisturl = "https://vba.dse.vic.gov.au/vba/downloadVSC.do" # gets a csv
vicchecklist = pd.read_csv(projectDir + "source-data/VIC/Species-Checklist.csv",encoding='cp1252')

# Extract and format initial list
1. Drop non-required columns
2. Rename columns for Darwin Core list terms
3. Assign additional required list fields

In [12]:
vicchecklist['LAST_MOD'].drop_duplicates().sort_values()

22182    01/01/2010
10432    01/02/2012
11265    01/02/2018
12493    01/02/2021
18298    01/03/2017
            ...    
13112    31/10/2016
159      31/10/2017
10501    31/10/2018
11206    31/10/2020
15567    31/10/2022
Name: LAST_MOD, Length: 899, dtype: object

In [13]:
# Drop columns
vicchecklist = vicchecklist.drop(['PARENT_TAXON_LEVEL_CDE','PARENT_TAXON_ID','ALL_DISCIPLINE_CODES',
                                          'TREATIES','SHORT_NAME','NVIS_GROWTHFORM','SCIENTIFIC_NME_SYNONYM',
                                          'COMMON_NME_SYNONYM','PRINT_ORDER_NUM', 'VIC_ADVISORY_STATUS', 'ORIGIN'],axis=1)
# Rename columns
vicchecklist = vicchecklist.rename(columns=
{
    'SCIENTIFIC_NAME':'scientificName',
    'FFG_ACT_STATUS':'ffgactstatus',
    'EPBC_ACT_STATUS': 'epbcactStatus',
    'TAXON_LEVEL_CDE': 'taxonRank',
    'ORIGIN': 'establishmentMeans',
    'COMMON_NME_SYNONYM': 'taxonRemarks',
    'COMMON_NAME':'vernacularName',
    'AUTHORITY':'scientificNameAuthority',
    'PRIMARY_DISCIPLINE': 'primaryDiscipline',
    'RESTRICTED_FLAG' : 'restrictedFlag',
    'EXTRACT_DATE': 'extractDate',
    'LAST_MOD':'modified'
})
vicchecklist['status'] = vicchecklist['ffgactstatus'].str.replace('(Extinct in Victoria)', '',  regex=False)
vicchecklist['taxonRank'] = 'species'
vicchecklist['status'] = vicchecklist['ffgactstatus']
vicchecklist['sourceStatus'] = vicchecklist['ffgactstatus']

# reformat date
vicchecklist['extractDate'] = pd.to_datetime(vicchecklist['extractDate'], format='%d/%m/%Y')
vicchecklist['extractDate'] = vicchecklist['extractDate'].dt.strftime('%Y-%m-%d')
vicchecklist['modified'] = pd.to_datetime(vicchecklist['modified'], format='%d/%m/%Y')
vicchecklist['modified'] = vicchecklist['modified'].dt.strftime('%Y-%m-%d')

In [14]:
vicchecklist['status'].unique()

array([nan, 'Extinct', 'Endangered', 'Critically Endangered',
       'Vulnerable', 'Threatened', 'Conservation Dependent',
       'Endangered (Extinct in Victoria)'], dtype=object)

In [15]:
vicchecklist['status'] = vicchecklist['status'].str.replace('(Extinct in Victoria)', '',  regex=False)
vicchecklist['status'] = vicchecklist['status'].str.strip()

# Split out conservation list

In [16]:
conservationList = vicchecklist[vicchecklist["status"].notna()]

c = [len(conservationList.index)]
d = {'conservation list count':[len(conservationList.index)]}
pd.DataFrame(d).transpose()

,0
sensitive list count,136
conservation list count,1999


## Process Conservation list

In [7]:
conservationList['status'].unique()

array(['Extinct', 'Endangered', 'Critically Endangered', 'Vulnerable',
       'Threatened', 'Conservation Dependent'], dtype=object)

In [8]:
conservationList.columns

Index(['TAXON_ID', 'scientificName', 'vernacularName',
       'scientificNameAuthority', 'primaryDiscipline', 'taxonRank',
       'TAXON_TYPE', 'ffgactstatus', 'epbcactStatus', 'restrictedFlag',
       'modified', 'extractDate', 'status', 'sourceStatus'],
      dtype='object')

In [9]:
conservationList= conservationList.loc[:, ['scientificName', 'vernacularName', 'status', 'sourceStatus']]

In [ ]:
#Write to CSV
dr = 'dr655'
conservationList.to_csv(projectDir + cdataDir + 'VIC-' + dr + '-conservation.csv',  encoding="UTF-8", index=False)
print("Finished writing VIC conservation list")

In [ ]:
### Change logs

In [ ]:
ltype = "C"
changeDir = "Monitoring/Change-logs/"
# VIC Conservation
print('VIC conservation')
filename = "VIC-conservation.csv"
testdr = "dr655"
proddr = "dr655"
changelist = lf.get_changelist(testdr, proddr, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
print('Finished VIC conservation')


In [ ]:
# Download historical lists from Production
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + proddr + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)  # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
print('Finished downloading conservation historical list')